In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
adnimerge=pd.read_csv('../data/ADNIMERGE_14Oct2024.csv')
unsam_data=pd.read_csv('../data/UNSAMparticipants.csv')

BAN_ADNI=pd.read_csv('../results/BrainAgeNeXt/BAN_ADNI.csv')
DBN_ADNI=pd.read_csv('../results/DeepBrainNet/DBN_ADNI_BET.csv')
pyment_ADNI=pd.read_csv('../results/pyment/pyment_ADNI.csv')
ENIGMA_ADNI=pd.read_csv('../results/ENIGMA/ENIGMA_ADNI.csv')

BAN_UNSAM=pd.read_csv('../results/BrainAgeNeXt/BAN_UNSAM.csv')
DBN_UNSAM=pd.read_csv('../results/DeepBrainNet/DBN_UNSAM.csv')
pyment_UNSAM=pd.read_csv('../results/pyment/pyment_UNSAM.csv')
ENIGMA_UNSAM=pd.read_csv('../results/ENIGMA/ENIGMA_UNSAM.csv')

C:\Users\parri\AppData\Local\Temp\ipykernel_24980\2697737962.py:1: DtypeWarning: Columns (19,20,21,50,51,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  adnimerge=pd.read_csv('../data/ADNIMERGE_14Oct2024.csv')


In [3]:
adnimerge=adnimerge[['PTID','PTRACCAT','PTEDUCAT','PTETHCAT']]
adnimerge=adnimerge[adnimerge['PTRACCAT']!='Am Indian/Alaskan']
unsam_data=unsam_data[['ID','NivelEducativo']]

In [4]:
#merge predictions with adnimerge
BAN_ADNI['PTID']=BAN_ADNI['ID'].str.split('_I').str[0]
DBN_ADNI['PTID']=DBN_ADNI['ID'].str.split('_I').str[0]
pyment_ADNI['PTID']=pyment_ADNI['ID'].str.split('_I').str[0]
ENIGMA_ADNI['PTID']=ENIGMA_ADNI['ID'].str.split('_I').str[0]

BAN_ADNI=pd.merge(BAN_ADNI,adnimerge,on='PTID', how='inner')
DBN_ADNI=pd.merge(DBN_ADNI,adnimerge,on='PTID',how='inner')
pyment_ADNI=pd.merge(pyment_ADNI,adnimerge,on='PTID',how='inner')
ENIGMA_ADNI=pd.merge(ENIGMA_ADNI,adnimerge,on='PTID',how='inner')

pyment_ADNI.drop_duplicates(subset=['ID'], keep='first', inplace=True,ignore_index=True)
DBN_ADNI.drop_duplicates(subset=['ID'], keep='first', inplace=True,ignore_index=True)
ENIGMA_ADNI.drop_duplicates(subset=['ID'], keep='first', inplace=True,ignore_index=True)
BAN_ADNI.drop_duplicates(subset=['ID'], keep='first', inplace=True,ignore_index=True)

In [5]:
#prepare dataframes for concatenation
BAN_ADNI=BAN_ADNI[['ID','Age','Error','Absolute_Error','PTRACCAT','PTEDUCAT','PTETHCAT','Group']]
DBN_ADNI.rename(columns={'Absolute_error':'Absolute_Error'}, inplace=True)
DBN_ADNI=DBN_ADNI[['ID','Age','Error','Absolute_Error','PTRACCAT','PTEDUCAT','PTETHCAT','Group']]
pyment_ADNI=pyment_ADNI[['ID','Age','Error','Absolute_Error','PTRACCAT','PTEDUCAT','PTETHCAT','Group']]
ENIGMA_ADNI.rename(columns={'Absolute Error':'Absolute_Error'}, inplace=True)
ENIGMA_ADNI=ENIGMA_ADNI[['ID','Age','Error','Absolute_Error','PTRACCAT','PTEDUCAT','PTETHCAT','Group' ]]

In [6]:
BAN_ADNI.loc[BAN_ADNI['PTETHCAT'] == 'Hisp/Latino', 'PTRACCAT'] = 'Hisp/Latino'
DBN_ADNI.loc[DBN_ADNI['PTETHCAT'] == 'Hisp/Latino', 'PTRACCAT'] = 'Hisp/Latino'
pyment_ADNI.loc[pyment_ADNI['PTETHCAT'] == 'Hisp/Latino', 'PTRACCAT'] = 'Hisp/Latino'
ENIGMA_ADNI.loc[ENIGMA_ADNI['PTETHCAT'] == 'Hisp/Latino', 'PTRACCAT'] = 'Hisp/Latino'

BAN_ADNI=BAN_ADNI[BAN_ADNI['PTRACCAT']!='More than one']
DBN_ADNI=DBN_ADNI[DBN_ADNI['PTRACCAT']!='More than one']
pyment_ADNI=pyment_ADNI[pyment_ADNI['PTRACCAT']!='More than one']
ENIGMA_ADNI=ENIGMA_ADNI[ENIGMA_ADNI['PTRACCAT']!='More than one']

In [7]:
ENIGMA_ADNI['PTRACCAT'].value_counts()

PTRACCAT
White          940
Black           82
Hisp/Latino     68
Asian           17
Name: count, dtype: int64

In [8]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio

In [9]:
datasets = {
    "ENIGMA": ENIGMA_ADNI[ENIGMA_ADNI['Group']=='CN'],
    "DeepBrainNet": DBN_ADNI[DBN_ADNI['Group']=='CN'],
    "Pyment": pyment_ADNI[pyment_ADNI['Group']=='CN'],
    "BrainAgeNeXt": BAN_ADNI[BAN_ADNI['Group']=='CN']
}
#ENIGMA_ADNI["PTRACCAT"] = pd.Categorical(ENIGMA_ADNI["PTRACCAT"], categories=["White", "Black", "Asian", "Hisp/Latino"], ordered=True) #para que no aparezca que el primer sujeto es asian
ethnicity_order = ["White", "Black", "Hisp/Latino", "Asian"]

# Crear figura con subplots 2x2
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=list(datasets.keys()),
    horizontal_spacing=0.025,
    vertical_spacing=0.09
)

axis_index = 1
row = 1
col = 1

for name, df in datasets.items():
    df = df.loc[df["PTRACCAT"].isin(ethnicity_order)].copy()
    df["PTRACCAT"] = pd.Categorical(df["PTRACCAT"], categories=["White", "Black", "Hisp/Latino", "Asian"], ordered=True)
    # Mediana del dataset actual
    global_median = df['Error'].median()
    medians = df.groupby("PTRACCAT")["Error"].median().reindex(ethnicity_order)

    Ns = {"White": 424, "Black": 62, "Asian": 13, "Hisp/Latino": 36}
    labels = [f"{e}<br>(N={Ns[e]})" for e in ethnicity_order]

    fig.add_trace(
        go.Box(
            x=df["PTRACCAT"],
            y=df["Error"],
            marker_color="red",
            boxmean=False,
            name=name,
            showlegend=False
        ),
        row=row, col=col
    )

    fig.add_trace(
        go.Scatter(
            x=ethnicity_order,
            y=medians.values,
            mode="text",
            text=[f"{m:.2f}" if not pd.isna(m) else "" for m in medians.values],
            textposition="top center",
            textfont=dict(size=16, color="black"),
            showlegend=False
        ),
        row=row, col=col
    )

    # --- LÍNEA PUNTEADA PARA ESTE SUBPLOT ---
    xref = f"x{axis_index} domain" if axis_index > 1 else "x domain"
    yref = f"y{axis_index}" if axis_index > 1 else "y"

    fig.add_shape(
        type="line",
        x0=0, x1=1,
        y0=global_median, y1=global_median,
        line=dict(color="black", width=2, dash="dash"),
        xref=xref,
        yref=yref
    )
    fig.add_trace(
        go.Scatter(
            x=[3.7],   
            y=[global_median+1],
            mode="text",
            text=[f"{global_median:.2f}"],
            textfont=dict(size=17, color="black"),
            showlegend=False,
        ),
        row=row, col=col
    )

    # Actualizar ejes
    fig.update_xaxes(
        ticktext=labels,
        tickvals=ethnicity_order,
        tickfont=dict(size=16),
        row=row, col=col,
        color='black'
    )

    if name in ['ENIGMA', 'Pyment']:
        
        fig.update_yaxes(title_text="BAG", row=row, col=col, color='black', title_font=dict(size=20))

    fig.update_yaxes(
        tickfont=dict(size=14),
        title_font=dict(size=18),
        showgrid=True, gridcolor='grey',
        zeroline=True, zerolinecolor='black',
        range=[-40, 20],
        row=row, col=col
    )

    # --- AVANZAR SUBPLOT ---
    col += 1
    if col == 3:
        col = 1
        row += 1

    axis_index += 1   # ← IMPORTANTE

# Ajuste general
fig.update_annotations(font_size=22)
fig.update_layout(
    height=800,
    width=1200,
    font=dict(size=14),
    margin=dict(l=20, r=20, t=30, b=10)
)

fig.show()


C:\Users\parri\AppData\Local\Temp\ipykernel_24980\2629899901.py:27: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\parri\AppData\Local\Temp\ipykernel_24980\2629899901.py:27: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\parri\AppData\Local\Temp\ipykernel_24980\2629899901.py:27: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\parri\AppData\Local\Temp\ipykernel_24980\2629899901.py:27: FutureWarning:

In [10]:
from scipy import stats

In [11]:

def t_test_error(df, group1, group2):
    group1_errors = df[df['PTRACCAT'] == group1]['Error']
    group2_errors = df[df['PTRACCAT'] == group2]['Error']
    t_stat, p_value = stats.ttest_ind(group1_errors, group2_errors, equal_var=False)
    return t_stat, p_value

datasets = {
    'df': BAN_ADNI[BAN_ADNI['Group']=='CN'], #se puede cambiar
}

results = {}
for name, df in datasets.items():
    ethnicity_groups = df['PTRACCAT'].unique()
    results[name] = {}
    for i, group1 in enumerate(ethnicity_groups):
        for group2 in ethnicity_groups[i+1:]:
            t_stat, p_value = t_test_error(df, group1, group2)
            results[name][f"{group1} vs {group2}"] = {'t_stat': t_stat, 'p_value': p_value}

# Show results
for name, comparisons in results.items():
    print(f"\n{name}:")
    for comparison, result in comparisons.items():
        print(f"  {comparison}: t-statistic = {result['t_stat']:.3f}, p-value = {result['p_value']:.3f}")


df:
  White vs Black: t-statistic = 2.608, p-value = 0.011
  White vs Hisp/Latino: t-statistic = -1.708, p-value = 0.094
  White vs Asian: t-statistic = -2.165, p-value = 0.050
  Black vs Hisp/Latino: t-statistic = -3.319, p-value = 0.001
  Black vs Asian: t-statistic = -3.199, p-value = 0.005
  Hisp/Latino vs Asian: t-statistic = -1.391, p-value = 0.184


In [12]:
DBN_ADNI.dropna(inplace=True)

In [13]:
#mean absolute error by PTRACCAT
print(f'White MAE',BAN_ADNI[(BAN_ADNI['PTRACCAT']=='White') & (BAN_ADNI['Group']=='CN')]['Absolute_Error'].mean())
print(f'Black MAE',BAN_ADNI[(BAN_ADNI['PTRACCAT']=='Black') & (BAN_ADNI['Group']=='CN')]['Absolute_Error'].mean())
print(f'Hisp/Latino',BAN_ADNI[(BAN_ADNI['PTRACCAT']=='Hisp/Latino') & (BAN_ADNI['Group']=='CN')]['Absolute_Error'].mean())
print(f'Asian MAE',BAN_ADNI[(BAN_ADNI['PTRACCAT']=='Asian') & (BAN_ADNI['Group']=='CN')]['Absolute_Error'].mean())


White MAE 4.020167605882477
Black MAE 5.520543110680957
Hisp/Latino 2.5743246782090923
Asian MAE 3.5796904719426044


In [14]:
import statsmodels.formula.api as smf
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.anova import anova_lm


In [15]:
BAN_CA_mean=np.mean(BAN_ADNI['Age'])
DBN_CA_mean=np.mean(DBN_ADNI['Age'])
pyment_CA_mean=np.mean(pyment_ADNI['Age'])
ENIGMA_CA_mean=np.mean(ENIGMA_ADNI['Age'])

BAN_ADNI['Corrected_age']=BAN_ADNI['Age']-BAN_CA_mean
DBN_ADNI['Corrected_age']=DBN_ADNI['Age']-DBN_CA_mean
pyment_ADNI['Corrected_age']=pyment_ADNI['Age']-pyment_CA_mean
ENIGMA_ADNI['Corrected_age']=ENIGMA_ADNI['Age']-ENIGMA_CA_mean

In [16]:
# Si tu dataframe se llama BAN_all:
model = smf.ols("Error ~ Corrected_age + C(PTRACCAT)", data=BAN_ADNI[BAN_ADNI['Group']=='CN']).fit()
model_reduced = smf.ols("Error ~ Corrected_age", data=BAN_ADNI[BAN_ADNI['Group']=='CN']).fit()

ancova_results =anova_lm(model,typ=2)
print(ancova_results)

# R² total 
r2_full = model.rsquared 

# Modelo reducido sin dataset (solo mean) 
r2_reduced = model_reduced.rsquared 

# f² para dataset 
f2_dataset = (r2_full - r2_reduced) / (1 - r2_full)
#0.02= small, 0.15=medium, 0.35=large 
print("f² (Dataset):", f2_dataset)

# Aplicar Tukey HSD
tukey = pairwise_tukeyhsd(endog=BAN_ADNI[BAN_ADNI['Group']=='CN']['Error'],      # variable dependiente
                          groups=BAN_ADNI[BAN_ADNI['Group']=='CN']['PTRACCAT'],  # variable categórica
                          alpha=0.05)

                    sum_sq     df          F        PR(>F)
C(PTRACCAT)     402.849918    3.0   7.697428  4.823488e-05
Corrected_age  1202.105056    1.0  68.907427  8.439813e-16
Residual       9350.636671  536.0        NaN           NaN
f² (Dataset): 0.043082619068664035


In [17]:
print(tukey)

     Multiple Comparison of Means - Tukey HSD, FWER=0.05      
   group1      group2   meandiff p-adj   lower   upper  reject
--------------------------------------------------------------
      Asian       Black  -4.5658 0.0043 -8.0459 -1.0857   True
      Asian Hisp/Latino  -1.8943 0.5502 -5.5909  1.8023  False
      Asian       White  -2.7952 0.1139 -6.0113   0.421  False
      Black Hisp/Latino   2.6715 0.0212  0.2847  5.0584   True
      Black       White   1.7706 0.0169  0.2293   3.312   True
Hisp/Latino       White  -0.9009 0.6453 -2.8832  1.0814  False
--------------------------------------------------------------


## education level influence on predictions

In [18]:
#BAN_UNSAM=BAN_UNSAM[BAN_UNSAM['Grupo']=="CONTROL"]
#DBN_UNSAM=DBN_UNSAM[DBN_UNSAM['Grupo']=="CONTROL"]
#pyment_UNSAM=pyment_UNSAM[pyment_UNSAM['Grupo']=="CONTROL"]
#ENIGMA_UNSAM=ENIGMA_UNSAM[ENIGMA_UNSAM['Grupo']=="CONTROL"]

BAN_ADNI['Group'] = BAN_ADNI['Group'].replace({'SMC': 'MCI', 'LMCI': 'MCI', 'EMCI': 'MCI'})
DBN_ADNI['Group'] = DBN_ADNI['Group'].replace({'SMC': 'MCI', 'LMCI': 'MCI', 'EMCI': 'MCI'})
pyment_ADNI['Group'] = pyment_ADNI['Group'].replace({'SMC': 'MCI', 'LMCI': 'MCI', 'EMCI': 'MCI'})
ENIGMA_ADNI['Group'] = ENIGMA_ADNI['Group'].replace({'SMC': 'MCI', 'LMCI': 'MCI', 'EMCI': 'MCI'})

In [19]:
unsam_data['NivelEducativo'].dropna(inplace=True)
BAN_UNSAM.rename(columns={'Predicted_Brain_Age':'prediction', 'Edad':'Age'},inplace=True)
BAN_UNSAM=pd.merge(BAN_UNSAM,unsam_data, on='ID', how='inner')

DBN_UNSAM.rename(columns={'Pred_Age':'prediction', 'Edad':'Age'},inplace=True)
DBN_UNSAM=pd.merge(DBN_UNSAM,unsam_data, on='ID', how='inner')

pyment_UNSAM.rename(columns={'Edad':'Age'},inplace=True)
pyment_UNSAM=pd.merge(pyment_UNSAM,unsam_data, on='ID', how='inner')

ENIGMA_UNSAM.rename(columns={'Edad':'Age','Absolute Error': 'Absolute_Error'},inplace=True)
ENIGMA_UNSAM=pd.merge(ENIGMA_UNSAM,unsam_data, on='ID', how='inner')

In [20]:
educ_map = {
    "Sin instrucción": 0,
    "Primario incompleto": 4,
    "Primario completo": 6,
    "Secundario incompleto": 9,
    "Secundario completo": 12,
    "Terciario incompleto": 13,
    "Terciario completo": 14,
    "Universitario incompleto": 14,
    "Universitario completo": 16,
    "Posgrado": 19
}

# Crear nueva columna
BAN_UNSAM["PTEDUCAT"] = BAN_UNSAM["NivelEducativo"].map(educ_map)
DBN_UNSAM["PTEDUCAT"] = DBN_UNSAM["NivelEducativo"].map(educ_map)
pyment_UNSAM["PTEDUCAT"] = pyment_UNSAM["NivelEducativo"].map(educ_map)
ENIGMA_UNSAM["PTEDUCAT"] = ENIGMA_UNSAM["NivelEducativo"].map(educ_map)


In [21]:
BAN_concat=pd.concat([BAN_ADNI,BAN_UNSAM],ignore_index=True)
DBN_concat=pd.concat([DBN_ADNI,DBN_UNSAM],ignore_index=True)
pyment_concat=pd.concat([pyment_ADNI,pyment_UNSAM],ignore_index=True)
ENIGMA_concat=pd.concat([ENIGMA_ADNI,ENIGMA_UNSAM],ignore_index=True)

In [22]:
BAN_concat=BAN_concat[['ID','Age','Error','Absolute_Error','PTEDUCAT']]
BAN_concat.dropna(inplace=True)
DBN_concat=DBN_concat[['ID','Age','Error','Absolute_Error','PTEDUCAT']]
DBN_concat.dropna(inplace=True)
pyment_concat=pyment_concat[['ID','Age','Error','Absolute_Error','PTEDUCAT']]
pyment_concat.dropna(inplace=True)
ENIGMA_concat=ENIGMA_concat[['ID','Age','Error','Absolute_Error','PTEDUCAT']]
ENIGMA_concat.dropna(inplace=True)

In [23]:
BAN_CA_mean=np.mean(BAN_concat['Age'])
DBN_CA_mean=np.mean(DBN_concat['Age'])
pyment_CA_mean=np.mean(pyment_concat['Age'])
ENIGMA_CA_mean=np.mean(ENIGMA_concat['Age'])

BAN_concat['Corrected_age']=BAN_concat['Age']-BAN_CA_mean
DBN_concat['Corrected_age']=DBN_concat['Age']-DBN_CA_mean
pyment_concat['Corrected_age']=pyment_concat['Age']-pyment_CA_mean
ENIGMA_concat['Corrected_age']=ENIGMA_concat['Age']-ENIGMA_CA_mean

In [24]:
BAN_UNSAM['Corrected_age']=BAN_UNSAM['Age']-np.mean(BAN_UNSAM['Age'])
DBN_UNSAM['Corrected_age']=DBN_UNSAM['Age']-np.mean(DBN_UNSAM['Age'])
pyment_UNSAM['Corrected_age']=pyment_UNSAM['Age']-np.mean(pyment_UNSAM['Age'])
ENIGMA_UNSAM['Corrected_age']=ENIGMA_UNSAM['Age']-np.mean(ENIGMA_UNSAM['Age'])

In [25]:
import statsmodels.formula.api as smf

In [26]:
BAN_UNSAM

,ID,prediction,Grupo,Age,Genero,Absolute_Error,Error,NivelEducativo,PTEDUCAT,Corrected_age
0,CP0053,53.864748,COVID,60,F,6.135252,-6.135252,Universitario completo,16.0,11.232673
1,CP0100,44.543446,COVID,50,M,5.456554,-5.456554,Secundario incompleto,9.0,1.232673
2,CP0154,43.123382,COVID,42,M,1.123382,1.123382,Terciario completo,14.0,-6.767327
3,CP0200,57.170476,CONTROL,64,F,6.829524,-6.829524,Secundario incompleto,9.0,15.232673
4,CP0001,29.173784,CONTROL,34,F,4.826216,-4.826216,Posgrado,19.0,-14.767327
...,...,...,...,...,...,...,...,...,...,...
197,CP0242,45.872900,COVID,45,M,0.872900,0.872900,Posgrado,19.0,-3.767327
198,CP0246,46.049595,CONTROL,51,M,4.950405,-4.950405,Universitario incompleto,14.0,2.232673
199,CP0247,42.377428,COVID,47,F,4.622572,-4.622572,Secundario incompleto,9.0,-1.767327
200,CP0249,30.394714,COVID,41,M,10.605286,-10.605286,Secundario incompleto,9.0,-7.767327


In [27]:
# Si tu dataframe se llama BAN_all:
model = smf.ols("Error ~ Corrected_age + PTEDUCAT + C(Grupo, Treatment(reference= 'CONTROL'))", data=BAN_UNSAM).fit()

model_reduced = smf.ols("Error ~ PTEDUCAT + C(Grupo, Treatment(reference= 'CONTROL'))", data=BAN_UNSAM).fit()

print(model.summary())

# R² total 
r2_full = model.rsquared 

# Modelo reducido sin dataset (solo mean) 
r2_reduced = model_reduced.rsquared 

# f² para dataset 
f2_reduced = (r2_full - r2_reduced) / (1 - r2_full)
#0.02= small, 0.15=medium, 0.35=large 
print("f² (variable):", f2_reduced)

                            OLS Regression Results                            
Dep. Variable:                  Error   R-squared:                       0.132
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     9.824
Date:                Tue, 16 Dec 2025   Prob (F-statistic):           4.61e-06
Time:                        22:57:16   Log-Likelihood:                -590.75
No. Observations:                 198   AIC:                             1189.
Df Residuals:                     194   BIC:                             1203.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------

In [30]:
# Promedio y desvío estándar de años de educación en ADNI y UNSAM

# ADNI
adni_mean = ENIGMA_ADNI['PTEDUCAT'].mean()
adni_std = ENIGMA_ADNI['PTEDUCAT'].std()

# UNSAM
unsam_mean = ENIGMA_UNSAM['PTEDUCAT'].mean()
unsam_std = ENIGMA_UNSAM['PTEDUCAT'].std()

print("Años de educación - ADNI:")
print(f"  Promedio: {adni_mean:.2f}")
print(f"  Desvío estándar: {adni_std:.2f}")
print(f"\nAños de educación - UNSAM:")
print(f"  Promedio: {unsam_mean:.2f}")
print(f"  Desvío estándar: {unsam_std:.2f}")

Años de educación - ADNI:
  Promedio: 16.38
  Desvío estándar: 2.47

Años de educación - UNSAM:
  Promedio: 13.51
  Desvío estándar: 2.97
